In [1]:
import pandas as pd
import geopandas as gpd
import json

In [2]:
requests = pd.read_csv("requests.csv")
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff
0,1380949200,1149,725,218,3601
1,1380949200,1225,1714,2003,3601
2,1380949200,1237,3143,1314,2218
3,1380949200,1323,3566,1330,2573
4,1380949200,1328,2466,1323,2473


In [3]:
requests['Hour'] = requests['Timestamp'].apply(lambda x: pd.Timestamp(x,unit='s',tz='America/New_York').hour)

In [4]:
requests['Serving'] = requests['Actual_Dropoff'].apply(lambda x: 0 if x ==-1 else 1 )

In [5]:
requests.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour,Serving
0,1380949200,1149,725,218,3601,1,1
1,1380949200,1225,1714,2003,3601,1,1
2,1380949200,1237,3143,1314,2218,1,1
3,1380949200,1323,3566,1330,2573,1,1
4,1380949200,1328,2466,1323,2473,1,1


In [6]:
unserving = requests[requests['Serving']==0]

In [7]:
unserving.head()

,Timestamp,Requested_Pickup,Requested_Dropoff,Actual_Pickup,Actual_Dropoff,Hour,Serving
688,1380949380,1225,1279,-1,-1,1,0
689,1380949380,1260,1999,-1,-1,1,0
692,1380949380,1387,1034,-1,-1,1,0
694,1380949380,1389,2569,-1,-1,1,0
709,1380949380,2013,1998,-1,-1,1,0


In [8]:
pick_up_location_by_time = unserving.groupby(['Hour','Requested_Pickup']).count()['Timestamp'].reset_index()

In [9]:
pick_up_location_by_time.rename(columns={'Timestamp':'Count'}, inplace=True)
pick_up_location_by_time.head()

,Hour,Requested_Pickup,Count
0,0,0,1
1,0,2,5
2,0,3,4
3,0,4,1
4,0,6,7


In [10]:
#pick_up_location_by_time.to_json(r'pick_up_location_by_time.json',orient='split')

In [11]:
#pick_up_location_by_time.to_csv(r'pick_up_location_by_time.csv')


In [12]:
servingrate = pd.read_csv("servingrate.csv")
servingrate.head()

,Unnamed: 0,Hour,Total,Serving,rate
0,0,0,19313,6748,0.349402
1,1,1,10052,7297,0.725925
2,2,2,6070,5464,0.900165
3,3,3,3725,3443,0.924295
4,4,4,2567,2310,0.899883


In [13]:
pick_up_location_by_time = pick_up_location_by_time.merge(servingrate, on="Hour")
pick_up_location_by_time.head()

,Hour,Requested_Pickup,Count,Unnamed: 0,Total,Serving,rate
0,0,0,1,0,19313,6748,0.349402
1,0,2,5,0,19313,6748,0.349402
2,0,3,4,0,19313,6748,0.349402
3,0,4,1,0,19313,6748,0.349402
4,0,6,7,0,19313,6748,0.349402


In [32]:
records = []
for key, grp in pick_up_location_by_time.groupby(['Hour','rate']):
    records.append({
        "hour": int(key[0]),
        "rate": float(key[1]),
        "perID": {
            row.Requested_Pickup: row.Count for row in grp.itertuples()
        }})

In [33]:
records

[{'hour': 0,
  'rate': 0.34940195723088074,
  'perID': {0: 1,
   2: 5,
   3: 4,
   4: 1,
   6: 7,
   8: 16,
   9: 18,
   10: 3,
   17: 2,
   20: 37,
   21: 11,
   22: 1,
   23: 4,
   24: 49,
   28: 1,
   31: 5,
   32: 2,
   36: 17,
   37: 33,
   47: 11,
   49: 8,
   50: 4,
   51: 3,
   53: 17,
   55: 2,
   56: 4,
   57: 5,
   59: 2,
   63: 1,
   65: 11,
   70: 1,
   71: 2,
   73: 11,
   74: 4,
   75: 1,
   76: 2,
   87: 2,
   94: 2,
   99: 2,
   101: 1,
   104: 2,
   106: 3,
   107: 2,
   108: 1,
   109: 2,
   112: 2,
   114: 16,
   147: 1,
   165: 2,
   169: 2,
   172: 15,
   173: 23,
   176: 1,
   178: 3,
   179: 6,
   180: 10,
   181: 7,
   182: 1,
   184: 4,
   191: 4,
   192: 1,
   194: 1,
   197: 7,
   199: 1,
   200: 3,
   207: 9,
   212: 6,
   213: 9,
   214: 4,
   215: 26,
   216: 11,
   217: 13,
   218: 6,
   219: 5,
   220: 1,
   221: 11,
   223: 13,
   225: 6,
   226: 3,
   227: 9,
   228: 2,
   229: 133,
   230: 27,
   231: 24,
   232: 9,
   233: 11,
   234: 13,
   235: 11

In [34]:
with open('pick_up_location_by_time.json', 'w',encoding='utf-8') as f:
     json.dump(records, f, ensure_ascii=False, indent=4)